In [2]:
import numpy as np
from ss_problem import SSProblem, SecureStateReconstruct
TS = 0.04
Ac = np.array([[0, 1, 0, 0],[0, -0.2, 0, 0],[0,0,0,1],[0,0,0,-0.2]])
Bc = np.array([[0, 0],[1, 0],[0, 0],[0,1]])
Cc = np.array([[1,0,0,0],[1,0,0,0],[0,1,0,0],[0,1,0,0],[0,0,1,0],[0,0,1,0],[0,0,0,1],[0,0,0,1]])
# Cc = np.vstack((Cc,Cc)) # if given enough sensor redundancy
Dc = np.zeros((Cc.shape[0],Bc.shape[1]))
s = 3

# generate discrete-time system
dtsys_a, dtsys_b, dtsys_c, dtsys_d = SSProblem.convert_ct_to_dt(Ac,Bc,Cc,Dc,TS)
# define input output data
init_state1 = np.array([[1.],[1.],[1.],[1.]])
init_state2 = np.array([[2.],[2.],[2.],[1.]])
# assume sensors 1,3,5 are under attack
sensor_initial_states = [init_state2,init_state1,init_state2,init_state1,
                            init_state2,init_state1,init_state1,init_state1]
u_seq, tilde_y_his, noise_level = SSProblem.generate_attack_measurement(dtsys_a, dtsys_b, dtsys_c, dtsys_d,sensor_initial_states,
                                                                        s = s,is_noisy =  False, noise_level=0.00001)

# construct a problem instance
ss_problem = SSProblem(dtsys_a, dtsys_b, dtsys_c, dtsys_d, tilde_y_his, 
                        attack_sensor_count=s,input_sequence=u_seq,measurement_noise_level= noise_level )
print(f'A: {ss_problem.A},  \n B: {ss_problem.B}, \n C:{ss_problem.C}, \n D:{ss_problem.D}')
print('input_sequence:',ss_problem.u_seq)
print('tilde_y_his:',ss_problem.tilde_y_his)

A: [[1.         0.03984043 0.         0.        ]
 [0.         0.99203191 0.         0.        ]
 [0.         0.         1.         0.03984043]
 [0.         0.         0.         0.99203191]],  
 B: [[0.00079787 0.        ]
 [0.03984043 0.        ]
 [0.         0.00079787]
 [0.         0.03984043]], 
 C:[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]], 
 D:[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
input_sequence: [[ 0.34095168  3.67971037]
 [ 2.94436452  3.89498753]
 [ 2.63151981 -3.66183829]
 [ 0.          0.        ]]
tilde_y_his: [[2.         1.         2.         1.         2.         1.
  1.         1.        ]
 [2.07995289 1.04011246 1.99764749 1.00561557 2.04277636 1.04277636
  1.13863314 1.13863314]
 [2.16188924 1.08252584 2.0990348  1.11490748 2.09124769 1.09124769
  1.28473838 1.28473838]
 [2.24761529 1.12904384 2.18715038 1.21086467 2.13951054 1.13951054
  1.12861228 1.1286

In [3]:
ssr_solution = SecureStateReconstruct(ss_problem)
# possible_states,corresp_sensors, _ = ssr_solution.solve(error_bound = 1)
possible_states,corresp_sensors, _ = ssr_solution.solve_initial_state(error_bound = 0.01)

[[[1.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]]

 [[1.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.03984043 0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]]

 [[1.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.0793634  0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]
  [0.         0.         0.         0. 

In [6]:
ssr_solution.obser[:,:,2]

array([[0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.99203191, 0.        , 0.        ],
       [0.        , 0.98412732, 0.        , 0.        ],
       [0.        , 0.97628571, 0.        , 0.        ]])

In [5]:
range(0)

range(0, 0)

In [7]:
print([i for i in range(0)])

[]


In [ ]:
%timeit possible_states,corresp_sensors, _ = ssr_solution.solve(error_bound = 1e-2)


In [ ]:
if possible_states is not None:
    for ind in range(corresp_sensors.shape[0]):
        sensors = corresp_sensors[ind,:]
        state = possible_states[:,ind]
        print(f'Identified possible states:{state} for sensors {sensors}')

In [ ]:
import timeit

execution_time = timeit.timeit(lambda: ssr_solution.solve(error_bound = 1e-2), number=100)

print(execution_time)


0.36949859996093437


In [ ]:
att_dic

{0: [1, 3, 5, 6, 7], 1: [0, 2, 4]}

In [ ]:
print(np.finfo(np.double).precision)

15


In [ ]:
residuals_list = [1,2,3,4]
residual_min = min(residuals_list)
comb_list = residual_min<2*residual_min

In [ ]:
residuals_list[0]

1